# <font color=#4267b2>用RNN作情意分析</font>
## <font color=#4267b2>設定及匯入需要的套件</font>

In [48]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [49]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

In [50]:
from keras.datasets import imdb

In [51]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

In [52]:
print('訓練總筆數:', len(x_train))
print('測試總筆數:', len(x_test))

訓練總筆數: 25000
測試總筆數: 25000


In [53]:
len(x_train[9487])

104

In [54]:
y_train[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int64)

In [55]:
y_train[24999]

0

In [56]:
from keras.preprocessing import sequence

In [140]:
x_train = sequence.pad_sequences(x_train, maxlen=200)
x_test = sequence.pad_sequences(x_test, maxlen=200)

In [141]:
x_train.shape

(25000, 200)

In [265]:
N = 10 # 文字要壓到 N 維
K = 20 # LSTM 有 K 個神經元

In [266]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=1, verbose=1)
from keras.optimizers import Nadam

In [267]:
model = Sequential()
model.add(Embedding(10000, N))
model.add(LSTM(K))
model.add(Dropout(0.5))
model.add(Dense(20, activation='selu'))
model.add(Dense(1, activation='sigmoid'))

In [269]:
optimizer_Na = Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
model.compile(loss='binary_crossentropy',
             optimizer=optimizer_Na,
             metrics=['accuracy'])

In [270]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_29 (Embedding)     (None, None, 10)          100000    
_________________________________________________________________
lstm_29 (LSTM)               (None, 20)                2480      
_________________________________________________________________
dropout_25 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_37 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_38 (Dense)             (None, 1)                 21        
Total params: 102,921
Trainable params: 102,921
Non-trainable params: 0
_________________________________________________________________


In [271]:
model.fit(x_train, y_train,
         batch_size=100,
         epochs=1, validation_data = (x_test,y_test), callbacks=[early_stopping])

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 55s 2ms/step - loss: 0.4688 - acc: 0.7622 - val_loss: 0.4345 - val_acc: 0.8075


In [272]:
model.fit(x_train, y_train,
         batch_size=100,
         epochs=1, validation_data = (x_test,y_test), callbacks=[early_stopping])

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 51s 2ms/step - loss: 0.2599 - acc: 0.8988 - val_loss: 0.3113 - val_acc: 0.8697


In [273]:
score = model.evaluate(x_test, y_test)

25000/25000 [==============================] - 19s 772us/step


In [274]:
print(f'測試資料的 loss = {score[0]}')
print(f'測試資正確率 = {score[1]}')

測試資料的 loss = 0.3113413042926788
測試資正確率 = 0.86972
